# Project EDA Starter Code

In [2]:
# Run this before any other code cell
# This downloads the data files into the same directory where you have saved this notebook

import urllib.request
from pathlib import Path
import os
path = Path()

# Dictionary of file names and download links
files = {'outage_data.parquet':'https://storage.googleapis.com/aipi_datasets/outage_data.parquet'}

# Download each file
for key,value in files.items():
    filename = path/key
    url = value
    # If the file does not already exist in the directory, download it
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url,filename)

outage_data.parquet


In [50]:
import pandas as pd
import numpy as np

In [51]:
# Read the data into a Pandas dataframe
df = pd.read_parquet(path='./outage_data.parquet', engine='pyarrow')

# Remove duplicate entries in 2019
# Remove all rows with SimStartDate after 2019-01-01 and event_type == 'thunderstorm'
df = df.loc[~((df['SimStartDate'] > '2019-01-01') & (df['event_type'] == 'thunderstorm'))]
df.describe()

,breaker_counts,fuse_counts,switch_counts,transformer_counts,recloser_counts,pole_counts,grid_id,outage_count,x,y,...,canopy_interval_71_80_60,canopy_interval_81_90_60,canopy_interval_91_100_60,canopy_mean_60,canopy_median_60,canopy_var_60,length_proxy_30,line_length_30,length_proxy_60,line_length_60
count,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,7.515200e+04,...,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,75152.000000,7.515200e+04,7.515200e+04,7.515200e+04,7.515200e+04
mean,2.153689,93.809426,24.516393,328.192623,6.491803,1155.500000,243.500000,0.489940,63859.779114,1.198521e+06,...,0.106869,0.034790,0.000189,29.837696,28.834016,453.072222,2.942549e+05,4.720946e+06,4.973358e+05,4.720946e+06
std,4.329085,86.394655,34.519465,307.756527,7.954581,1075.414373,140.874107,1.356861,57254.344305,1.524949e+04,...,0.116302,0.064809,0.001851,19.703477,24.602453,263.134070,2.628181e+05,4.422564e+06,4.090464e+05,4.422564e+06
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-28082.000000,1.165209e+06,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.160000e+02,0.000000e+00,1.106000e+03,0.000000e+00
25%,0.000000,21.000000,2.000000,64.000000,0.000000,288.000000,121.750000,0.000000,16918.000000,1.186893e+06,...,0.004917,0.000000,0.000000,11.905220,0.000000,251.758973,8.346350e+04,1.169986e+06,1.532600e+05,1.169986e+06
50%,0.000000,73.000000,11.000000,237.000000,3.000000,885.500000,243.500000,0.000000,52918.000000,1.201209e+06,...,0.062440,0.005809,0.000000,31.113005,26.000000,506.095082,2.260190e+05,3.405802e+06,4.117480e+05,3.405802e+06
75%,2.000000,141.250000,34.000000,512.500000,10.000000,1663.750000,365.250000,0.000000,103918.000000,1.210209e+06,...,0.173633,0.036777,0.000000,46.485635,47.000000,638.942526,4.424790e+05,7.083961e+06,7.661815e+05,7.083961e+06
max,36.000000,450.000000,281.000000,1291.000000,45.000000,4724.000000,487.000000,48.000000,262365.000000,1.237893e+06,...,0.500000,0.641975,0.037846,78.185185,83.000000,1305.809392,1.056230e+06,1.762393e+07,1.508426e+06,1.762393e+07


In [4]:
# Save feature names to a csv file
df.columns.to_frame().to_csv('features.csv', index=False)

In [5]:
# Count the number of grid cells and outage events
# 488 grid cells and 154 outage events
df.loc[:,['lat','lon']].groupby(['lat','lon']).size().reset_index(name='count').sort_values(by='count', ascending=False)

,lat,lon,count
0,40.546875,-73.921875,154
335,40.921875,-73.265625,154
333,40.921875,-73.328125,154
332,40.921875,-73.359375,154
331,40.921875,-73.390625,154
...,...,...,...
158,40.796875,-73.703125,154
157,40.796875,-73.734375,154
156,40.796875,-73.765625,154
155,40.765625,-72.703125,154


In [6]:
# Sort events by number of outages
grouped = df.loc[:,['SimStartDate','event_type','outage_count']].groupby(['SimStartDate','event_type']).outage_count.sum().sort_values(ascending=False)
grouped = grouped.reset_index(name='outage_count')
grouped = grouped.set_index('SimStartDate')
grouped

,event_type,outage_count
SimStartDate,,
2018-03-02 00:00:00+00:00,rainwind,1969.0
2016-02-24 00:00:00+00:00,thunderstorm,1369.0
2017-01-23 00:00:00+00:00,rainwind,824.0
2018-10-27 00:00:00+00:00,rainwind,710.0
2019-02-24 09:00:00+00:00,rainwind,656.0
...,...,...
2016-03-16 00:00:00+00:00,thunderstorm,97.0
2016-04-26 00:00:00+00:00,thunderstorm,96.0
2016-02-21 00:00:00+00:00,rainwind,91.0


In [7]:
# Sort events by SimStartDate
grouped = df.loc[:,['SimStartDate','event_type','outage_count']].groupby(['SimStartDate','event_type']).outage_count.sum().sort_index()
grouped = grouped.reset_index(name='outage_count')
# save to csv
grouped.to_csv('outage_count.csv')

In [11]:
df['event_type'].value_counts()

rainwind        48312
thunderstorm    26840
Name: event_type, dtype: int64